In [8]:
%matplotlib notebook 
# Import modules
import os
import math
import numpy as np
import matplotlib.pyplot
from pyne import serpent
from pyne import nucname

ace_lib_path = '/home/andrei2/serpent/xsdata/jeff312'
acelib = "sss_jeff312.xsdata"
#dep = serpent.parse_dep('../second_case/flux/TMSRPu_dep.m', make_mats=False)
dep = serpent.parse_dep('../third_case/flux/TMSRTRU_dep.m', make_mats=False)
#res = serpent.parse_res('../serpent/60-years-analysis/TMSR_res.m')
#res2 = serpent.parse_res('/home/andrei2/Desktop/git/msr-neutronics/depletion/online_1200days_Pa_less_Th/less_Th_rate/core_res.m')

target_iso = 'U233'

plot_title = 'Pa outflow 1.25E-10, U-233 inflow 3.5E-9, Th 9E-9\n'
days = dep['DAYS']                        # Time array parsed from *_dep.m file
bu = dep['MAT_fuel_BURNUP']                        # Time array parsed from *_dep.m file
time_step = np.diff(days)                 # Depletion time step evaluation
names = dep['NAMES'][0].split()                      # Names of isotopes parsed from *_dep.m file
#keff_analytical = res['ANA_KEFF']         # K-eff parsing from dictionary
#keff_analytical2 = res2['ANA_KEFF']
#kinf_dict = res['ABS_KINF']               # K-inf parsing from dictionary
#kinf_dict2 = res2['ABS_KINF']
#bu = res['BURNUP']                        # Burnup parsing from dictionary
#keff_a = keff_analytical[:,0]             # K-eff value
#keff_error = keff_analytical[:,1]         # K-eff standart deviation
#kinf = kinf_dict[:,0]
#burnup = bu[:,0]
EOC = np.amax(days)                       # End of cycle (simulation time length)
#total_mass_list = dep['TOT_MASS']
adens_fuel = dep['MAT_fuel_ADENS']        # atomic density for each isotope in material 'fuel'
mdens_fuel = dep['MAT_fuel_MDENS']        # mass density for each isotope in material 'fuel'
vol_fuel = dep['MAT_fuel_VOLUME']         # total volume of material 'fuel'
density_fuel = mdens_fuel[-1,:]           # total mass density for material 'fuel'

print(dep.keys()) #prints keys     

dict_keys(['i531361', 'i581520', 'i330780', 'MAT_fuel_BURNUP', 'i410940', 'i531290', 'i400970', 'i40080', 'i551320', 'i481150', 'i691720', 'i531350', 'i932350', 'i701720', 'i541351', 'i501320', 'i451200', 'i451030', 'i571390', 'i481111', 'i491200', 'i531300', 'i651700', 'i521270', 'i491291', 'i501240', 'i421100', 'i481131', 'i491120', 'i461130', 'i531331', 'i350860', 'i501311', 'i651590', 'i591430', 'i681710', 'i922390', 'MAT_pa_tank_VOLUME', 'i421000', 'i451050', 'i491280', 'i922360', 'i90200', 'i481230', 'i521360', 'i922330', 'i501251', 'i952440', 'i962410', 'i451051', 'i330830', 'i471140', 'i531360', 'i561470', 'i471181', 'i280660', 'i822120', 'i410951', 'i421030', 'ZAI', 'i391040', 'MAT_fuel_A', 'i942370', 'BU', 'i631620', 'i461071', 'i561310', 'i571440', 'i511270', 'i320840', 'i661600', 'i401020', 'i611470', 'i651660', 'i471170', 'i320830', 'i621630', 'i410970', 'i561420', 'i330800', 'i651670', 'i982490', 'MAT_add_GSRC', 'i300711', 'i551380', 'i511301', 'i300790', 'i481090', 'i430

In [9]:
def get_library_isotopes():
    """ Returns the isotopes in the cross section library
    Parameters:
    -----------
    Returns:
    --------
    iso_array: array
        array of isotopes in cross section library:
    """
    lib_isos = []
    # check if environment variable is set
    if os.environ.get('SERPENT_DATA') is not None:
        path =  os.environ['SERPENT_DATA']
    else:
        path = ace_lib_path
    acelib_path = path +"/sss_jeff312.xsdata"
    
    with open(acelib_path, 'r') as f:
        metaflag = 0
        lines = f.readlines()
        for line in lines:
            iso = (line.split()[1]).split('.')[0]
            metaflag = line.split()[4]
            zzaaa = line.split()[3]
            if metaflag == '1':
                iso = iso.replace(iso, zzaaa)
            iso = str(iso) + str(metaflag)
            lib_isos.append(iso)
            
    return np.array(lib_isos)
    

#print (days)
print (days[21])

#print (bu)

#print(dep['iLOST'])
#print(adens_fuel[223-1])
#print(mdens_fuel[223-1])
print(nucname.serpent(952421))

#print(mdens_fuel[-1,:])

moment = -1

lib_isotopes = get_library_isotopes()

matf = open('mat_comp', 'w')
matf.write('mat fuel -%7.9E  tmp 900  vol %7.5E\n' % (density_fuel[moment],
                                                     vol_fuel[moment]))
for key, value in dep.items():
    if key[0]=='i':
        if key[1:] not in lib_isotopes:
            matf.write('%15s       -%7.9E\n' % (key[1:], mdens_fuel[value-1,moment]))
        elif key[1:] in lib_isotopes:
            matf.write('%11s.09c       -%7.9E\n' % (nucname.serpent(key[1:]), mdens_fuel[value-1,moment]))
        else:
            raise ValueError('Wrong isotope name')
        #print (mdens_fuel[value-2, 0])
        
matf.close()

7665.0
Am-242m
